<a href="https://colab.research.google.com/github/anazaidah/DASBOARD-SENTIMEN-ANALYSIS/blob/main/Aspect_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TOPIK MODELLING LDA 

In [ ]:
import nltk 
import nltk.data
import json
import pandas as pd
import numpy as np
import re
import sys
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
!pip install python-rake
import RAKE
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import collections
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# read data
df = pd.read_csv("/content/reviews.csv", usecols = ['content']);
data = df.loc [:,"content"]


In [ ]:
# case text as lowercase, remove punctuation, remove extra whitespace in string and on both sides of string

df['remove_lower_punct'] = df['content'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()
#
display(df.head(10))

,content,remove_lower_punct
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...
5,Tidak bisa masuk. Saya sudah mengaktifkan GPS ...,tidak bisa masuk saya sudah mengaktifkan gps l...
6,Setiap saya coba buka aplikasi ini selalu ada ...,setiap saya coba buka aplikasi ini selalu ada ...
7,"Saya tidak bisa mengakses aplikasi tge, setela...",saya tidak bisa mengakses aplikasi tge setelah...
8,"Tanggal Lahir memang sangat menyebalkan, kita ...",tanggal lahir memang sangat menyebalkan kita h...
9,"Aplikasi tidak berguna, jangan buang data selu...",aplikasi tidak berguna jangan buang data selul...


In [ ]:
# tokenise string
df['tokenise'] = df.apply(lambda row: nltk.word_tokenize(row[1]), axis=1)
display(df.head(10))


,content,remove_lower_punct,tokenise
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...,"[aplikasi, selalu, menampilkan, pemberitahuan,..."
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...,"[tidak, berguna, untuk, dipasang, selalu, munc..."
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...,"[mereka, minta, dicarikan, tiket, vaksinasi, y..."
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...,"[aplikasi, yang, sangat, buruk, saya, tidak, b..."
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...,"[saya, menginstal, ini, dan, dikatakan, ponsel..."
5,Tidak bisa masuk. Saya sudah mengaktifkan GPS ...,tidak bisa masuk saya sudah mengaktifkan gps l...,"[tidak, bisa, masuk, saya, sudah, mengaktifkan..."
6,Setiap saya coba buka aplikasi ini selalu ada ...,setiap saya coba buka aplikasi ini selalu ada ...,"[setiap, saya, coba, buka, aplikasi, ini, sela..."
7,"Saya tidak bisa mengakses aplikasi tge, setela...",saya tidak bisa mengakses aplikasi tge setelah...,"[saya, tidak, bisa, mengakses, aplikasi, tge, ..."
8,"Tanggal Lahir memang sangat menyebalkan, kita ...",tanggal lahir memang sangat menyebalkan kita h...,"[tanggal, lahir, memang, sangat, menyebalkan, ..."
9,"Aplikasi tidak berguna, jangan buang data selu...",aplikasi tidak berguna jangan buang data selul...,"[aplikasi, tidak, berguna, jangan, buang, data..."


In [ ]:
# initiate stopwords from nltk
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

# remove stopwords
df['remove_stopwords'] = df['tokenise'].apply(lambda x: [item for item in x if item not in stopwords])
display(df.head(10))

,content,remove_lower_punct,tokenise,remove_stopwords
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...,"[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,..."
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...,"[tidak, berguna, untuk, dipasang, selalu, munc...","[berguna, dipasang, selalu, muncul, notifikasi..."
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...,"[mereka, minta, dicarikan, tiket, vaksinasi, y...","[minta, dicarikan, tiket, vaksinasi, fiturnya,..."
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...,"[aplikasi, yang, sangat, buruk, saya, tidak, b...","[aplikasi, sangat, buruk, masuk, aplikasi, kat..."
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...,"[saya, menginstal, ini, dan, dikatakan, ponsel...","[menginstal, dikatakan, ponsel, root, jailbrok..."
5,Tidak bisa masuk. Saya sudah mengaktifkan GPS ...,tidak bisa masuk saya sudah mengaktifkan gps l...,"[tidak, bisa, masuk, saya, sudah, mengaktifkan...","[masuk, mengaktifkan, gps, lokasi, diperlukan,..."
6,Setiap saya coba buka aplikasi ini selalu ada ...,setiap saya coba buka aplikasi ini selalu ada ...,"[setiap, saya, coba, buka, aplikasi, ini, sela...","[coba, buka, aplikasi, selalu, notifikasi, kon..."
7,"Saya tidak bisa mengakses aplikasi tge, setela...",saya tidak bisa mengakses aplikasi tge setelah...,"[saya, tidak, bisa, mengakses, aplikasi, tge, ...","[mengakses, aplikasi, tge, memperbarui, versi,..."
8,"Tanggal Lahir memang sangat menyebalkan, kita ...",tanggal lahir memang sangat menyebalkan kita h...,"[tanggal, lahir, memang, sangat, menyebalkan, ...","[tanggal, lahir, memang, sangat, menyebalkan, ..."
9,"Aplikasi tidak berguna, jangan buang data selu...",aplikasi tidak berguna jangan buang data selul...,"[aplikasi, tidak, berguna, jangan, buang, data...","[aplikasi, berguna, jangan, buang, data, selul..."


In [ ]:
# initiate nltk lemmatiser
wordnet_lemmatizer = WordNetLemmatizer()

# lemmatise words
df['lemmatise'] = df['remove_stopwords'].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x]) 

display(df.head(10))

,content,remove_lower_punct,tokenise,remove_stopwords,lemmatise
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...,"[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,..."
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...,"[tidak, berguna, untuk, dipasang, selalu, munc...","[berguna, dipasang, selalu, muncul, notifikasi...","[berguna, dipasang, selalu, muncul, notifikasi..."
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...,"[mereka, minta, dicarikan, tiket, vaksinasi, y...","[minta, dicarikan, tiket, vaksinasi, fiturnya,...","[minta, dicarikan, tiket, vaksinasi, fiturnya,..."
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...,"[aplikasi, yang, sangat, buruk, saya, tidak, b...","[aplikasi, sangat, buruk, masuk, aplikasi, kat...","[aplikasi, sangat, buruk, masuk, aplikasi, kat..."
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...,"[saya, menginstal, ini, dan, dikatakan, ponsel...","[menginstal, dikatakan, ponsel, root, jailbrok...","[menginstal, dikatakan, ponsel, root, jailbrok..."
5,Tidak bisa masuk. Saya sudah mengaktifkan GPS ...,tidak bisa masuk saya sudah mengaktifkan gps l...,"[tidak, bisa, masuk, saya, sudah, mengaktifkan...","[masuk, mengaktifkan, gps, lokasi, diperlukan,...","[masuk, mengaktifkan, gps, lokasi, diperlukan,..."
6,Setiap saya coba buka aplikasi ini selalu ada ...,setiap saya coba buka aplikasi ini selalu ada ...,"[setiap, saya, coba, buka, aplikasi, ini, sela...","[coba, buka, aplikasi, selalu, notifikasi, kon...","[coba, buka, aplikasi, selalu, notifikasi, kon..."
7,"Saya tidak bisa mengakses aplikasi tge, setela...",saya tidak bisa mengakses aplikasi tge setelah...,"[saya, tidak, bisa, mengakses, aplikasi, tge, ...","[mengakses, aplikasi, tge, memperbarui, versi,...","[mengakses, aplikasi, tge, memperbarui, versi,..."
8,"Tanggal Lahir memang sangat menyebalkan, kita ...",tanggal lahir memang sangat menyebalkan kita h...,"[tanggal, lahir, memang, sangat, menyebalkan, ...","[tanggal, lahir, memang, sangat, menyebalkan, ...","[tanggal, lahir, memang, sangat, menyebalkan, ..."
9,"Aplikasi tidak berguna, jangan buang data selu...",aplikasi tidak berguna jangan buang data selul...,"[aplikasi, tidak, berguna, jangan, buang, data...","[aplikasi, berguna, jangan, buang, data, selul...","[aplikasi, berguna, jangan, buang, data, selul..."


In [ ]:
# initialise the count vectorizer

vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
                            
# join the processed data to be vectorised

vectors = []

for index, row in df.iterrows():
    vectors.append(", ".join(row[4]))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)

  (0, 611)	1
  (0, 7447)	1
  (0, 5132)	1
  (0, 6420)	1
  (0, 6786)	1
  (0, 8845)	1
  (0, 4376)	1
  (0, 3622)	1
  (0, 451)	1
  (0, 7153)	1
  (0, 528)	1
  (0, 3846)	1
  (0, 1464)	1
  (0, 1468)	1
  (0, 9223)	1
  (0, 6545)	1
  (0, 5137)	1
  (0, 593)	1
  (0, 6511)	1
  (1, 4376)	1
  (1, 1262)	1
  (1, 2327)	1
  (1, 7452)	1
  (1, 5540)	1
  (1, 5904)	1
  :	:
  (1193, 9056)	1
  (1193, 1703)	1
  (1193, 3524)	1
  (1193, 1894)	1
  (1194, 7575)	1
  (1194, 2485)	1
  (1194, 7727)	1
  (1194, 7220)	1
  (1194, 3533)	1
  (1194, 8915)	1
  (1194, 8165)	1
  (1194, 9064)	1
  (1194, 3956)	1
  (1194, 9511)	1
  (1194, 5687)	1
  (1194, 7558)	1
  (1196, 4179)	1
  (1196, 3807)	1
  (1196, 656)	1
  (1196, 7617)	1
  (1196, 6262)	1
  (1197, 5508)	1
  (1198, 7224)	1
  (1198, 5049)	1
  (1198, 4968)	1


In [ ]:
# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 5, # number of topics
                                  random_state = 10,    # random state
                                  evaluate_every = -1,  # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,           # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic

# join to original dataframes

df = pd.merge(df, df_document_topic, left_index = True, right_index = True, how = 'outer')
display(df.head(10))

,content,remove_lower_punct,tokenise,remove_stopwords,lemmatise,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...,"[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,...",0.01,0.01,0.96,0.01,0.01,3
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...,"[tidak, berguna, untuk, dipasang, selalu, munc...","[berguna, dipasang, selalu, muncul, notifikasi...","[berguna, dipasang, selalu, muncul, notifikasi...",0.01,0.01,0.01,0.01,0.96,5
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...,"[mereka, minta, dicarikan, tiket, vaksinasi, y...","[minta, dicarikan, tiket, vaksinasi, fiturnya,...","[minta, dicarikan, tiket, vaksinasi, fiturnya,...",0.02,0.93,0.02,0.02,0.02,2
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...,"[aplikasi, yang, sangat, buruk, saya, tidak, b...","[aplikasi, sangat, buruk, masuk, aplikasi, kat...","[aplikasi, sangat, buruk, masuk, aplikasi, kat...",0.01,0.01,0.01,0.01,0.96,5
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...,"[saya, menginstal, ini, dan, dikatakan, ponsel...","[menginstal, dikatakan, ponsel, root, jailbrok...","[menginstal, dikatakan, ponsel, root, jailbrok...",0.02,0.93,0.02,0.02,0.02,2
5,Tidak bisa masuk. Saya sudah mengaktifkan GPS ...,tidak bisa masuk saya sudah mengaktifkan gps l...,"[tidak, bisa, masuk, saya, sudah, mengaktifkan...","[masuk, mengaktifkan, gps, lokasi, diperlukan,...","[masuk, mengaktifkan, gps, lokasi, diperlukan,...",0.03,0.03,0.03,0.03,0.89,5
6,Setiap saya coba buka aplikasi ini selalu ada ...,setiap saya coba buka aplikasi ini selalu ada ...,"[setiap, saya, coba, buka, aplikasi, ini, sela...","[coba, buka, aplikasi, selalu, notifikasi, kon...","[coba, buka, aplikasi, selalu, notifikasi, kon...",0.02,0.92,0.02,0.02,0.02,2
7,"Saya tidak bisa mengakses aplikasi tge, setela...",saya tidak bisa mengakses aplikasi tge setelah...,"[saya, tidak, bisa, mengakses, aplikasi, tge, ...","[mengakses, aplikasi, tge, memperbarui, versi,...","[mengakses, aplikasi, tge, memperbarui, versi,...",0.01,0.96,0.01,0.01,0.01,2
8,"Tanggal Lahir memang sangat menyebalkan, kita ...",tanggal lahir memang sangat menyebalkan kita h...,"[tanggal, lahir, memang, sangat, menyebalkan, ...","[tanggal, lahir, memang, sangat, menyebalkan, ...","[tanggal, lahir, memang, sangat, menyebalkan, ...",0.01,0.01,0.96,0.01,0.01,3
9,"Aplikasi tidak berguna, jangan buang data selu...",aplikasi tidak berguna jangan buang data selul...,"[aplikasi, tidak, berguna, jangan, buang, data...","[aplikasi, berguna, jangan, buang, data, selul...","[aplikasi, berguna, jangan, buang, data, selul...",0.01,0.01,0.01,0.95,0.01,4


In [ ]:
# apply sentiment analysis
analyser = SentimentIntensityAnalyzer()

sentiment_score_list = []
sentiment_label_list = []

for i in df['remove_lower_punct'].values.tolist():
    sentiment_score = analyser.polarity_scores(i)

    if sentiment_score['compound'] >= 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Positif')
    elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('negatif')
    elif sentiment_score['compound'] <= -0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('netral')
    
df['sentiment'] = sentiment_label_list
df['sentiment score'] = sentiment_score_list

display(df.head(900))

,content,remove_lower_punct,tokenise,remove_stopwords,lemmatise,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic,sentiment,sentiment score
0,Aplikasi selalu menampilkan pemberitahuan pop-...,aplikasi selalu menampilkan pemberitahuan pop ...,"[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,...","[aplikasi, selalu, menampilkan, pemberitahuan,...",0.01,0.01,0.96,0.01,0.01,3,negatif,0.0000
1,Tidak berguna untuk dipasang. Selalu muncul no...,tidak berguna untuk dipasang selalu muncul not...,"[tidak, berguna, untuk, dipasang, selalu, munc...","[berguna, dipasang, selalu, muncul, notifikasi...","[berguna, dipasang, selalu, muncul, notifikasi...",0.01,0.01,0.01,0.01,0.96,5,negatif,0.0000
2,"Mereka minta dicarikan Tiket Vaksinasi, ya ada...",mereka minta dicarikan tiket vaksinasi ya ada ...,"[mereka, minta, dicarikan, tiket, vaksinasi, y...","[minta, dicarikan, tiket, vaksinasi, fiturnya,...","[minta, dicarikan, tiket, vaksinasi, fiturnya,...",0.02,0.93,0.02,0.02,0.02,2,negatif,0.0000
3,"Aplikasi yang sangat buruk, saya tidak bisa ma...",aplikasi yang sangat buruk saya tidak bisa mas...,"[aplikasi, yang, sangat, buruk, saya, tidak, b...","[aplikasi, sangat, buruk, masuk, aplikasi, kat...","[aplikasi, sangat, buruk, masuk, aplikasi, kat...",0.01,0.01,0.01,0.01,0.96,5,negatif,0.0000
4,Saya menginstal ini dan dikatakan ponsel saya ...,saya menginstal ini dan dikatakan ponsel saya ...,"[saya, menginstal, ini, dan, dikatakan, ponsel...","[menginstal, dikatakan, ponsel, root, jailbrok...","[menginstal, dikatakan, ponsel, root, jailbrok...",0.02,0.93,0.02,0.02,0.02,2,negatif,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Well Secured,well secured,"[well, secured]","[well, secured]","[well, secured]",0.10,0.10,0.10,0.60,0.10,4,Positif,0.5859
896,Masih keder,masih keder,"[masih, keder]",[keder],[keder],0.20,0.20,0.20,0.20,0.20,1,negatif,0.0000
897,Ok,ok,[ok],[],[],0.20,0.20,0.20,0.20,0.20,1,Positif,0.2960
898,Okay,okay,[okay],[okay],[okay],0.20,0.20,0.20,0.20,0.20,1,Positif,0.2263


In [ ]:
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topic_names

df_topic_no = pd.DataFrame(df_topic_keywords.idxmax())
df_scores = pd.DataFrame(df_topic_keywords.max())

tmp = pd.merge(df_topic_no, df_scores, left_index=True, right_index=True)
tmp.columns = ['topic', 'relevance_score']

display(tmp)

,topic,relevance_score
19 mei,Topic3,1.199996
a5 android,Topic5,1.199993
a5s apr,Topic4,1.199989
abal2 paling,Topic2,1.199990
abis update,Topic2,2.200552
...,...,...
zone what,Topic1,1.200016
zone while,Topic5,2.199998
zone yet,Topic5,1.199997
zone zone,Topic2,1.199997


In [ ]:
keywords = np.array(vectorizer.get_feature_names())

topic_keywords = []

for topic_weights in lda_model.components_:
    top_keyword_locs = (-topic_weights).argsort()[:5]
    topic_keywords.append(keywords.take(top_keyword_locs))

for i in topic_keywords:
    print(i)

['this app' 'the app' 'user friendly' 'tanggal lahir' 'terima kasih']
['sertifikat vaksin' 'koneksi internet' 'update malah' 'vaksin pertama'
 'the app']
['sertifikat vaksin' 'jailbreak root' 'force close' 'this app'
 'root padahal']
['the app' 'this app' 'to use' 'easy to' 'it keep']
['koneksi internet' 'sertifikat vaksin' 'root jailbreak' 'the app'
 'red zone']


In [ ]:
all_topics = []

for i in tmp['topic'].unique():    
    tmp_1 = tmp.loc[tmp['topic'] == i].reset_index()
    tmp_1 = tmp_1.sort_values('relevance_score', ascending=False).head(1)

    tmp_1['topic'] = tmp_1['topic'] 
    
    tmp_2 = []
    tmp_2.append(tmp_1['topic'].unique()[0])
    tmp_2.append(list(tmp_1['index'].unique()))
    all_topics.append(tmp_2)

all_topics = pd.DataFrame(all_topics, columns=['Dominant_topic','topic_name'])
display(all_topics)

,Dominant_topic,topic_name
0,Topic3,[sertifikat vaksin]
1,Topic5,[koneksi internet]
2,Topic4,[the app]
3,Topic2,[update malah]
4,Topic1,[user friendly]


In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   content             1200 non-null   object 
 1   remove_lower_punct  1200 non-null   object 
 2   tokenise            1200 non-null   object 
 3   remove_stopwords    1200 non-null   object 
 4   lemmatise           1200 non-null   object 
 5   Topic1              1200 non-null   float64
 6   Topic2              1200 non-null   float64
 7   Topic3              1200 non-null   float64
 8   Topic4              1200 non-null   float64
 9   Topic5              1200 non-null   float64
 10  Dominant_topic      1200 non-null   int64  
 11  sentiment           1200 non-null   object 
 12  sentiment score     1200 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 171.2+ KB


SENTIMEN VADER

In [321]:
results = df.groupby(['Dominant_topic', 'sentiment']).count().reset_index()

results = results.merge(all_topics, on='Dominant_topic')
results['topic_name'] = results['topic_name'].apply(', '.join)

graph_results = results[['topic_name', 'sentiment', 'sentiment score']]
graph_results = graph_results.pivot(index='topic_name', columns='sentiment', values='sentiment score').reset_index()

graph_results.set_index('topic_name', inplace=True)

display(graph_results)

ValueError: ignored